<a href="https://colab.research.google.com/github/OseiasBeu/Data-Science/blob/master/Analisando_Base_do_AirBNB_com_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [12]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

# download do http para arquivo local
!wget --show-progress https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/listings.csv

--2021-11-15 22:22:47--  https://raw.githubusercontent.com/OseiasBeu/Data-Science/master/DataSets/listings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3298987 (3.1M) [text/plain]
Saving to: ‘listings.csv’

listings.csv        100%[===================>]   3.15M  --.-KB/s    in 0.02s   

2021-11-15 22:22:47 (146 MB/s) - ‘listings.csv’ saved [3298987/3298987]



### Summary information and metrics for listings in Rio de Janeiro (good for visualisations).
### 26 October, 2021
### Rio de Janeiro

In [16]:
listings = sc.read.load('./listings.csv', headers=True, inferSchema=True, format='csv',sep=',')
listings.show()

+-----+--------------------+---------+--------------------+-------------------+---------------+---------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+--------------------+----------------+--------------------+-------+
|  _c0|                 _c1|      _c2|                 _c3|                _c4|            _c5|      _c6|      _c7|            _c8|  _c9|          _c10|             _c11|       _c12|             _c13|                _c14|            _c15|                _c16|   _c17|
+-----+--------------------+---------+--------------------+-------------------+---------------+---------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+--------------------+----------------+--------------------+-------+
|   id|                name|  host_id|           host_name|neighbourhood_group|  neighbourhood| latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_mo

In [17]:
type(listings)

pyspark.sql.dataframe.DataFrame

In [18]:
listings.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17']

In [19]:
listings.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)



In [38]:
from functools import reduce
from past.builtins import xrange

cols = []
for columnName in listings.first():
  cols.append(columnName)
oldColumns = listings.schema.names

print(f'''
Novas colunas: {cols}\n
Velhas colunas: {oldColumns}
''')
df = reduce(lambda listings, idx: listings.withColumnRenamed(oldColumns[idx], cols[idx]), xrange(len(oldColumns)), listings)
df.printSchema()
df.show()


Novas colunas: ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license']

Velhas colunas: ['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17']

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable 

id
